In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
sys.path.insert(0, "..")

In [2]:
import trimesh
import matplotlib
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt

# Visualize the Implicit Field

In [3]:
!ls ../pretrained/neural_fields/

armadillo  bunny   cylinder  half_noisy_sphere	noisy_torus
bar	   cactus  dino      jolteon


In [4]:
log_path = "../pretrained/neural_fields/armadillo"

In [5]:
import yaml
import argparse
import os.path as osp
from train import dict2namespace
from pprint import pprint
from utils import load_imf

original, cfg = load_imf(
    log_path, 
    config_fpath=osp.join(log_path, "config.yaml"), 
    return_cfg=True)
original

../utils.py:51: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = dict2namespace(yaml.load(f))


../pretrained/neural_fields/armadillo/latest.pt
Net:
Net(
  (blocks): ModuleList(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): Linear(in_features=512, out_features=1, bias=True)
  )
  (act): Sine()
)


Net(
  (blocks): ModuleList(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): Linear(in_features=512, out_features=1, bias=True)
  )
  (act): Sine()
)

In [6]:
import numpy as np
import torch
from tqdm import tqdm
import skimage
import skimage.measure
from trainers.utils.vis_utils import imf2mesh
    
original.eval()
res = 256
new_mesh = imf2mesh(original, res=res, threshold=0.00, normalize=True, norm_type='res')
new_mesh

100%|██████████| 1678/1678 [00:06<00:00, 253.47it/s]


<trimesh.Trimesh(vertices.shape=(74745, 3), faces.shape=(149490, 3))>

In [7]:
# verts = (new_mesh.vertices * 2 - res) / float(res)
# new_mesh = trimesh.Trimesh(vertices=verts, faces=new_mesh.faces)
print(new_mesh.vertices.max(), new_mesh.vertices.min())
mp.plot(new_mesh.vertices, new_mesh.faces)

0.5956034660339355 -0.7738829851150513


/home/grendelyang/anaconda3/envs/IGP/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/grendelyang/anaconda3/envs/IGP/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0216128…

# Creating Handles

1. Creating a set of deforming handles (i.e. sparse points on the surface that will change according to the shape)
2. Creating a set of statisc handles (i.e. sparse points on the surface that will remain unchanged)

In [8]:
verts, faces = new_mesh.vertices, new_mesh.faces
print(verts.max(axis=0))
print(verts.min(axis=0))

[0.59560347 0.5954181  0.50229275]
[-0.55237782 -0.77388299 -0.5414533 ]


In [99]:
right_hand = verts[(np.abs(verts[:, 1] - 0.39) < 0.0025) & 
                  (np.abs(verts[:, 2] + 0.225) < 0.0025) & 
                  (verts[:, 0] > 0.5), :][0:1, :]
print(right_hand.shape)
left_hand = verts[
    (np.abs(verts[:, 1] - 0.345) < 0.0025) & 
    (np.abs(verts[:, 2] + 0.35) < 0.005) &
    (verts[:, 0] < -0.53)
    , :][0:1, :]
print(left_hand.shape)
left_foot = verts[
    (verts[:, 1] < -0.76) & (verts[:, 0] < 0)
    , :
]
right_foot = verts[
    (verts[:, 1] < -0.76) & (verts[:, 0] > 0)
    , :
]
p = mp.plot(new_mesh.vertices, new_mesh.faces)
p.add_points(right_hand, shading={"point_color": "blue", "point_size": 0.1})
p.add_points(left_hand, shading={"point_color": "red", "point_size": 0.1})
p.add_points(left_foot, shading={"point_color": "green", "point_size": 0.1})
p.add_points(right_foot, shading={"point_color": "black", "point_size": 0.1})

(1, 3)
(1, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0216128…

4

In [123]:
from scipy.spatial.transform import Rotation as ROT

def rot_and_translate(pts, rot_axis=None, rot_deg=None, translate=None):
    center = pts.mean(axis=0).reshape(1, 3)
    if rot_axis is None or rot_deg is None:
        rot_mat = np.eye(3)
    else:
        if isinstance(rot_axis, list) and isinstance(rot_deg, list):
            rot_mat = np.eye(3)
            for i, (ra, rd) in enumerate(zip(rot_axis, rot_deg)):
                print(i, ra,rd)
                rot_mat_curr = ROT.from_euler(ra, rd, degrees=True).as_matrix()
                rot_mat = np.matmul(rot_mat, rot_mat_curr)
        else:
            rot_mat = ROT.from_euler(rot_axis, rot_deg, degrees=True).as_matrix()
    if translate is None:
        translate = np.array([0, 0, 0])
    else:
        translate = np.array(translate).reshape(1, 3)
    transf = np.matmul(pts - center, rot_mat) + center + translate
    return transf

# pts = left_front
# pts = nose
pts = left_hand
pts_out = rot_and_translate(pts, translate=np.array([0.2, 0.4, 0.1]))

pts = right_hand
pts_out = rot_and_translate(pts, translate=np.array([-0.2, 0.4, -0.1]))

pts = left_foot
pts_out = rot_and_translate(pts, translate=np.array([0.25, 0.25, 0]), 
                            rot_axis=['y', 'z'], rot_deg=[-45, -90])


p = mp.plot(new_mesh.vertices, new_mesh.faces)
p.add_points(pts, shading={'point_color': 'green', 'point_size': 0.1})
p.add_points(pts_out, shading={"point_color": "blue", "point_size": 0.1})

print(pts.shape, pts_out.shape)

0 y -45
1 z -90


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0216128…

(904, 3) (904, 3)


In [127]:
handles, targets = [], []
for h, r_axis, r_angle, translate in [
    (left_hand, None, None, [0.2, 0.4, 0.1]),
    (right_hand, None, None, [-0.2, 0.4, -0.1]),
    (left_foot, ['y', 'z'], [-45, -90], [0.25, 0.25, 0]),
    (right_foot, None, None, None)
]:
    handles.append(h.reshape(-1, 3))
    targets.append(rot_and_translate(h, rot_axis=r_axis, rot_deg=r_angle, translate=translate))
    
handles = np.concatenate(handles, axis=0)
targets = np.concatenate(targets, axis=0)

p = mp.plot(new_mesh.vertices, new_mesh.faces)
p.add_points(handles, shading={"point_color": "blue", "point_size": 0.1})
p.add_points(targets, shading={"point_color": "red", "point_size": 0.1})

0 y -45
1 z -90


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0216128…

2

# Compare to Open3D ASAP Algorithm

In [128]:
from trainers.utils.o3d_deformation import deform_mesh_o3d, trimesh_to_o3dmesh, o3dmesh_to_trimesh
mesh_prime_tri = deform_mesh_o3d(original, handles, targets, imf_mesh = new_mesh, 
                                 steps=100, smoothed_alpha=100)

Smoothing alphas: 100 Use Smoothed Energy
[Open3D DEBUG] [DeformAsRigidAsPossible] setting up S'
[Open3D DEBUG] [DeformAsRigidAsPossible] done setting up S'
[Open3D DEBUG] [DeformAsRigidAsPossible] setting up system matrix L
[Open3D DEBUG] [DeformAsRigidAsPossible] done setting up system matrix L
[Open3D DEBUG] [DeformAsRigidAsPossible] setting up sparse solver
[Open3D DEBUG] [DeformAsRigidAsPossible] done setting up sparse solver
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=0, energy=8.994552e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=1, energy=8.994552e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=2, energy=8.994551e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=3, energy=8.994551e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=4, energy=8.994550e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=5, energy=8.994549e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=6, energy=8.994549e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=7, energy=8.994548e-01
[Open3D DEBUG

In [129]:
p = mp.plot(mesh_prime_tri.vertices, mesh_prime_tri.faces)
p.add_points(handles, shading={"point_color": "blue", "point_size": 0.05})
p.add_points(targets, shading={"point_color": "red", "point_size": 0.05})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0444710…

2

In [132]:
from trainers.utils.o3d_deformation import deform_mesh_o3d, trimesh_to_o3dmesh, o3dmesh_to_trimesh
mesh_prime_tri = deform_mesh_o3d(original, handles, targets, imf_mesh = new_mesh, 
                                 steps=10000, smoothed_alpha=10)


Smoothing alphas: 10 Use Smoothed Energy
[Open3D DEBUG] [DeformAsRigidAsPossible] setting up S'
[Open3D DEBUG] [DeformAsRigidAsPossible] done setting up S'
[Open3D DEBUG] [DeformAsRigidAsPossible] setting up system matrix L
[Open3D DEBUG] [DeformAsRigidAsPossible] done setting up system matrix L
[Open3D DEBUG] [DeformAsRigidAsPossible] setting up sparse solver
[Open3D DEBUG] [DeformAsRigidAsPossible] done setting up sparse solver
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=0, energy=8.994552e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=1, energy=8.994547e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=2, energy=8.994540e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=3, energy=8.994534e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=4, energy=8.994528e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=5, energy=8.994523e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=6, energy=8.994517e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=7, energy=8.994511e-01
[Open3D DEBUG]

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=112, energy=8.994088e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=113, energy=8.994084e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=114, energy=8.994081e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=115, energy=8.994078e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=116, energy=8.994074e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=117, energy=8.994071e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=118, energy=8.994068e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=119, energy=8.994065e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=120, energy=8.994061e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=121, energy=8.994058e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=122, energy=8.994055e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=123, energy=8.994052e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=124, energy=8.994048e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=125, energy=8.994045e-01
[Open3

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=229, energy=8.993740e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=230, energy=8.993737e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=231, energy=8.993734e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=232, energy=8.993732e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=233, energy=8.993729e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=234, energy=8.993726e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=235, energy=8.993724e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=236, energy=8.993721e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=237, energy=8.993718e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=238, energy=8.993716e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=239, energy=8.993713e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=240, energy=8.993710e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=241, energy=8.993708e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=242, energy=8.993705e-01
[Open3

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=346, energy=8.993447e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=347, energy=8.993445e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=348, energy=8.993442e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=349, energy=8.993440e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=350, energy=8.993438e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=351, energy=8.993436e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=352, energy=8.993433e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=353, energy=8.993431e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=354, energy=8.993429e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=355, energy=8.993426e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=356, energy=8.993424e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=357, energy=8.993422e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=358, energy=8.993419e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=359, energy=8.993417e-01
[Open3

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=465, energy=8.993192e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=466, energy=8.993190e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=467, energy=8.993188e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=468, energy=8.993186e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=469, energy=8.993184e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=470, energy=8.993182e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=471, energy=8.993180e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=472, energy=8.993178e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=473, energy=8.993176e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=474, energy=8.993174e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=475, energy=8.993172e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=476, energy=8.993170e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=477, energy=8.993168e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=478, energy=8.993166e-01
[Open3

...

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9492, energy=9.003639e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9493, energy=9.003642e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9494, energy=9.003644e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9495, energy=9.003646e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9496, energy=9.003649e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9497, energy=9.003651e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9498, energy=9.003653e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9499, energy=9.003656e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9500, energy=9.003658e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9501, energy=9.003660e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9502, energy=9.003663e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9503, energy=9.003665e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9504, energy=9.003667e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9505, energy=9.003

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9611, energy=9.003920e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9612, energy=9.003923e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9613, energy=9.003925e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9614, energy=9.003927e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9615, energy=9.003930e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9616, energy=9.003932e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9617, energy=9.003934e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9618, energy=9.003937e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9619, energy=9.003939e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9620, energy=9.003941e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9621, energy=9.003944e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9622, energy=9.003946e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9623, energy=9.003949e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9624, energy=9.003

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9728, energy=9.004198e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9729, energy=9.004201e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9730, energy=9.004203e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9731, energy=9.004205e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9732, energy=9.004208e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9733, energy=9.004210e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9734, energy=9.004212e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9735, energy=9.004215e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9736, energy=9.004217e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9737, energy=9.004220e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9738, energy=9.004222e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9739, energy=9.004224e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9740, energy=9.004227e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9741, energy=9.004

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9844, energy=9.004475e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9845, energy=9.004478e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9846, energy=9.004480e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9847, energy=9.004483e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9848, energy=9.004485e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9849, energy=9.004487e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9850, energy=9.004490e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9851, energy=9.004492e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9852, energy=9.004495e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9853, energy=9.004497e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9854, energy=9.004499e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9855, energy=9.004502e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9856, energy=9.004504e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9857, energy=9.004

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9962, energy=9.004759e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9963, energy=9.004762e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9964, energy=9.004764e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9965, energy=9.004766e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9966, energy=9.004769e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9967, energy=9.004771e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9968, energy=9.004774e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9969, energy=9.004776e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9970, energy=9.004778e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9971, energy=9.004781e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9972, energy=9.004783e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9973, energy=9.004786e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9974, energy=9.004788e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=9975, energy=9.004

In [133]:
p = mp.plot(mesh_prime_tri.vertices, mesh_prime_tri.faces)
p.add_points(handles, shading={"point_color": "blue", "point_size": 0.05})
p.add_points(targets, shading={"point_color": "red", "point_size": 0.05})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0444035…

2

# Save the Data for Neural Fields Deformation

In [134]:
# Saving it to data
out_dir = "../data/armadillo/deform_ballet.npy"
np.save(out_dir, {
    "handles": handles,
    "targets": targets,
    'gtr_verts': mesh_prime_tri.vertices,
    'gtr_faces': mesh_prime_tri.faces
})
out_dir

'../data/armadillo/deform_ballet.npy'